# 215 - Crack-free Walls

## Problem Statement

Consider the problem of building a wall out of $2 \times 1$ and $3 \times 1$ bricks ($\text{horizontal} \times \text{vertical}$ dimensions) such that, for extra strength, the gaps between horizontally-adjacent bricks never line up in consecutive layers, i.e. never form a "running crack".

For example, the following $9 \times 3$ wall is not acceptable due to the running crack shown in red:

<div class="center">
<img src="images/0215_crackfree.gif" style="background-color: white;" alt="0215_crackfree">
</div>

There are eight ways of forming a crack-free $9 \times 3$ wall, written $W(9,3) = 8$.

Calculate $W(32,10)$.

## Solution

We generate all the possible combinations walls of unit height using a simple recursive function. Then for each combination, we check the number of other combinations that are compatible. For a given combination at a given layer, we sum the number of configurations leading to compatible combinations at the previous layer. 

In [1]:
from collections import defaultdict
from functools import cache
import numpy as np
from numba import njit

In [2]:
solutions = []
n = 32
m = 10

def generate_solutions(solution):
    if sum(solution) == n:
        solutions.append(np.cumsum(solution.copy()))
    elif sum(solution) > n:
        return
    solution.append(2)
    generate_solutions(solution)
    solution.pop()
    solution.append(3)
    generate_solutions(solution)
    solution.pop()

generate_solutions([])

In [3]:
@njit
def check_compatibility_nb(arr1, arr2):
    i = 0
    j = 0
    while i < len(arr1) - 1 and j < len(arr2) - 1:
        if arr1[i] == arr2[j]:
            return False
        elif arr1[i] < arr2[j]:
            i += 1
        else:
            j += 1
    return True


@cache
def check_compatibility(i, j):
    return check_compatibility_nb(solutions[i], solutions[j])


mapping = {}
for sol in solutions:
    mapping[tuple(sol)] = 1

for _ in range(m-1):
    prev_mapping = mapping
    mapping = defaultdict(int)
    for i in range(len(solutions) - 1):
        for j in range(i + 1, len(solutions)):
            if check_compatibility(i, j):
                mapping[tuple(solutions[i])] += prev_mapping[tuple(solutions[j])]
                mapping[tuple(solutions[j])] += prev_mapping[tuple(solutions[i])]


In [4]:
sum(mapping.values())

806844323190414